Copyright <2024> <Carlos A. Ariza - cariza@alum.mit.edu>

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [ ]:
# Key requirement: Installing PDFTK Server
# PDFTK Server: CLI for pdftk
# Install it at:
# https://www.pdflabs.com/tools/pdftk-server/
# Available for Mac and Windows.
# For Macs running M1 or higher, it needs this version: https://www.pdflabs.com/tools/pdftk-the-pdf-toolkit/pdftk_server-2.02-mac_osx-10.11-setup.pkg


In [ ]:
# pip install pypdftk

In [ ]:
# pip install usaddress                # used to parse addresses into components

In [ ]:
# pip install reportlab              # used to generate PDF continuation sheet

In [1]:
import os
import pypdftk
import numpy as np
import pandas as pd
import usaddress
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import Paragraph
from reportlab.lib import colors
from reportlab.lib.units import inch
from datetime import date

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
os.getcwd()

In [ ]:
# Define path for source CSV file: 
csv_path = 'Springfield_0 3.csv'

In [ ]:
os.listdir()

In [ ]:
# Load data file as a pandas dataframe:
input_all = pd.read_csv(csv_path)

In [ ]:
display(input_all)

In [ ]:
# BREAK UP ADDRESS FIELD INTO MULTIPLE FIELDS AS EXPECTED BY THE PDF FORM

decomposed_addresses = input_all['Address'].apply(lambda x: usaddress.tag(str(x))[0])
print(decomposed_addresses)

# Convert the decomposed addresses into a DataFrame
decomposed_df = pd.DataFrame(decomposed_addresses.tolist())

# Concatenate the decomposed DataFrame with the original DataFrame
input_all = pd.concat([input_all, decomposed_df], axis=1)



In [ ]:
for i in range(111):
    print(input_all.columns[i])

In [ ]:

# Rename columns
new_columns = {
                'Site Number':'SiteID',
                'Field Date':'FieldDate',

                'Is this an original (new) form?':'Original',
                'Site Name':'Sitename',
                'Survey Project Name':'ProjectName',
                'National Register Category':'NRcategory',
                'Ownership':'Ownership',
                'Township':'Township1',
                'Range':'Range1',
                'Section':'Section1',
                'Tax Parcel #':'TaxParcel',
                'Subdivision Name':'Subdivision',
                'Block':'Block',
                'Lot':'Lot',
                'x':'CoordX',
                'y':'CoordY',
                'Construction Year':'YrBuilt',
                'apprx':'YrBuiltQualifier',
                'Original Use':'OriginalUse',
                'From (year)':'OrigUseFromYear',
                'To (year)':'OrigUseToYear',
                'Current Use':'CurrentUse',
                'From (year).1':'CurrUseFromYear',
                'To (year).1':'CurrUseToYear',
                'Other Use':'OtherUse',
                'From (year).2':'OtherUseFromYear',
                'To (year).2':'OtherUseToYear',
                'Move':'Moves',
                'if_yes_date':'MoveDate',
                'original_address':'OriginalAddress',
                'Any Alterations?':'Alterations',
                'Date of Alterations':'AlterationDate',
                'Description of alterations':'NatureAlteration',
                'Any Additions?':'Additions',
                'Date of Additions':'AdditionDate',
                'Description of additions':'NatureAddition',
                'Architect (last name first)':'Architect',
                'Builder (last name first)':'Builder',
                'Ownership History':'OwnershipHistory',
                'Is this resource affected by a local ordinance?':'PresOrdinance',
                'Primary Building Style':'Style',
                'Exterior Plan':'ExteriorPlan',
                'Number of Stories':'Stories',
                'Primary Exterior Fabric':'ExteriorFabric1',
                'Secondary Exterior Fabric':'ExteriorFabric2',
                'Tertiary Exterior Fabric':'ExteriorFabric3',
                'Primary Roof Type':'RoofType1',
                'Secondary Roof Type':'RoofType2',
                'Tertiary Roof Type':'RoofType3',
                'Primary Roof Material':'RoofMaterial1',
                'Secondary Roof Material':'RoofMaterial2',
                'Tertiary Roof Material':'RoofMaterial3',
                'Secondary Roof Structure (1)':'RoofSecondaryStruc1',
                'Secondary Roof Structure (2)':'RoofSecondaryStruc2',
                'Windows':'Windows',
                'Distinguishing Architectural Features':'DistinguishingFeatures',
                'Ancillary Features or Outbuilldings':'AncillaryFeatures',
                'Number of Chimneys':'ChimneyNo',
                'Chimney Material 1':'ChimneyMaterials1',
                'Chimney Material 2':'ChimneyMaterials2',
                'Primary Structural System':'StrucSys1',
                'Secondary Structural System':'StrucSys2',
                'Primary Foundation Type':'FoundationType1',
                'Secondary Foundation Type':'FoundationType2',
                'Foundation Material 1':'FoundationMatl1',
                'Foundation Material 1.1':'FoundationMatl2',
                'Main Entrance':'MainEntrance',
                'Porch Description':'PorchDescriptions',
                'Condition of Resource':'Condition',
                'Narrative Description of Resource':'NarrativeDescription',
                'Archaeological Remains':'ArchaeologicalRemains',
                'Reseach_methods_used':'ResMethods', # this will be expanded into individual fields later
                'Other Methods':'OtherResearchMeth',
                'Bibliographic References':'BibliographicRef',
                'Appears to meet the criteria for National Register listing individually?':'SurvEvalInd',
                'Appears to meet the criteria for National Register listing as part of a district?':'SurvEvalDist',
                'Explanation of Evaluation':'ExplanationEvaluation',
                'Areas of Significance.1':'HistAssociation1',
                'Areas of Significance.2':'HistAssociation2',
                'Areas of Significance.3':'HistAssociation3',
                'Areas of Significance.4':'HistAssociation4',
                'Areas of Significance.5':'HistAssociation5',
                'Areas of Significance.6':'HistAssociation6',
                'Document Type':'RepositoryCat1',
                'Maintaining Organization':'RepositoryOrg1',
                'Recorder_name':'RecorderName',
                'Affiliation':'RecorderAffiliation',
                'Recorder Contact Information':'RecorderContactInfo',
                'AddressNumber': 'StreetNumber',
                'StreetNamePreDirectional': 'StreetDirection',
                'StreetName': 'StreetName',
                'StreetNamePostType': 'StreetType',
                'StreetNamePostDirectional': 'SuffixDirection',
                'PlaceName':'City'
               }
input_all.rename(columns=new_columns, inplace=True)

In [ ]:
# Convert the Original field to all uppercase
input_all['Original'] = input_all['Original'].str.upper()

In [ ]:
# lookup street types to match the form drop down

strtyp_dict = {
   'Avenue':'AVE',
'Boulevard':'BLVD',
'Causeway':'CAUS',
'Circle':'CIRC',
'Concourse':'CONC',
'Court':'CT',
'Drive':'DR',
'Highway':'HWY',
'Lane':'LN',
'Loop':'LOOP',
'No Formal Address':'NFA',
'Not applicable':'NA',
'Parkway':'PKWY',
'Place':'PL',
'Plaza':'PLAZ',
'Point':'PT',
'Road':'RD',
'Rural Route':'RURL',
'Square':'SQ',
'Street':'ST',
'Terrace':'TERR',
'Trace':'TRAC',
'Trail':'TRAL',
'Unknown to recorder':'UNKN',
'Unspcified by recrdr':'UNSP',
'Walk':'WALK',
'Way':'WAY',
'Ave':'AVE',
'Blvd':'BLVD',
'Cir':'CIRC',
'Ct':'CT',
'Dr':'DR',
'Ln':'LN',
'Pkwy':'PKWY',
'Pl':'PL',
'Rd':'RD',
'St':'ST',
'Sq':'SQ',
'Ter':'TERR',
'Way':'WAY',
'Trl':'TRAL',
'Plz':'PLAZ',
'Hwy':'HWY',
'Walk':'WALK',
'Cswy':'CAUS',
'Pt':'PT',
}


#Replace StreetType text with codes:
input_all['StreetType'].replace(strtyp_dict, inplace=True)

In [ ]:
# Force InCity to always be YES

input_all['InCity'] = 'YES'

In [ ]:
# Form Date is the date in which the form was filled out: Today                             

input_all['FormDate']  = date.today()

In [ ]:
input_all['FieldDate'] = pd.to_datetime(input_all['FieldDate'])  # Convert to datetime

# Extract date and update the DataFrame
input_all['FieldDate'] = input_all['FieldDate'].dt.strftime('%m-%d-%Y')

input_all['FormDate'] = pd.to_datetime(input_all['FormDate'])  # Convert to datetime

# Extract date and update the DataFrame
input_all['FormDate'] = input_all['FormDate'].dt.strftime('%m-%d-%Y')

In [ ]:
display(input_all)

In [ ]:
# Replace NaN with blanks: use fillna()

input_all.fillna('', inplace=True)

In [ ]:
# Uppercase YES NO fields:                                                                     #####################################

# List of fields to upper case:
fields_to_capitalize = ['SiteID','Original', 'Moves', 'Alterations', 'Additions','PresOrdinance','SurvEvalInd','SurvEvalDist']  # 'InCity', needs to be added to dataframe

# Upper case text in specified columns:
input_all[fields_to_capitalize] = input_all[fields_to_capitalize].apply(lambda x: x.str.upper())

In [ ]:
# Lookup table: Ownership

own_dict = {
    'private_profit':'CORP',  'private_nonprofit':'CONP', 'private_individual':'PRIV',   'city':'CITY',   'private_nonspecific':'PRUN', 'county':'COUN' ,'state':'STAT', 
    'federal':'FEDE', 'Native_American':'NAAM', 'foreign':'FORE', 'unknown':'UNKN'
}

#Replace Ownership text with codes:
input_all['Ownership'].replace(own_dict, inplace=True)

In [ ]:
display(input_all)

In [ ]:
#County: Inferred from SiteID

def extract_first_two_chars(text):
     if len(text) >= 2:
         return text[:2].upper()
     else:
        return ''
# Create new column with county abbreviation
input_all['County'] = input_all['SiteID'].apply(extract_first_two_chars)



In [ ]:
#Lookup table: Original Use, Current Use, Other Use

use_dict = {
   'Abandoned/Vacant':'VACA',
'Agricultural':'AGRI',
'Air/Bus/Rail terminal or depot':'TERM',
'Airport':'AIRP',
'Animal shelter/Kennel/Pound':'KENN',
'Apartment':'APTM',
'Apartment, garage':'GAPT',
'Apartments & commercial':'CMAP',
'Arched entryway':'GATE',
'Armory':'ARMO',
'Air Force/Army/Navy/Military base':'MILI',
'Art gallery/Museum/Planetarium':'MUSE',
'Auto dealership':'AUTO',
'Auto repair/Gas station':'GAST',
'Bakery':'BAKE',
'Band shell':'BDSH',
'Bank':'BANK',
'Bar':'BAR',
'Barber/Salon/Beauty shop':'SBEA',
'Barn':'BARN',
'Beauty shop/Salon/Barber':'SBEA',
'Bed & Breakfast':'BEDB',
'Belltower/Clocktower/Tower':'TOWR',
'Blacksmith/Smithy':'SBLK',
'Blockhouse/Fortification':'FORT',
'Boardinghouse':'ROOM',
'Boat Ramp':'BOMP',
'Boat Slip':'SLIP',
'Boathouse':'BOAT',
'Boatworks':'BOAW',
'Bridge':'BRID',
'Brothel':'HPRO',
'Bus terminal':'TERM',
'Cabin':'RESI',
'Cafeteria/Restaurant/Diner':'REST',
'Cafeteria, not retail':'DINI',
'Cemetery':'CEME',
'Cemetery/Church Complex':'CHCE',
'Charnel house':'CHAR',
'Child care':'CHIL',
'Children\'s home/Orphanage':'ORPH',
'Church/Temple/Synagogue':'HSEW',
'Church/Cemetery Complex':'CHCE',
'City hall':'CIHL',
'Civic center':'CIVI',
'Clinic/Health center/Outpatient care':'OUTP',
'Clocktower/Belltower/Tower':'TOWR',
'Clubhouse':'CLUB',
'College/University/School':'SCHO',
'Commercial & apartments':'CMAP',
'Commercial & residence':'CMRE',
'Commercial':'COMM',
'Communications-related':'CMUN',
'Community center':'BREC',
'Control tower':'CTOW',
'Convent/Monastery':'CONV',
'Correctional facilty':'JAIL',
'Cottage':'RESI',
'Courthouse':'COUR',
'Credit union':'BANK',
'Dairy':'DAIR',
'Dam':'DAM',
'Day care':'CHIL',
'Demolished/Destroyed':'DEST',
'Dentist/Medical/Professional office':'OFFI',
'Department store':'SDEP',
'Depot':'TERM',
'Destroyed/Demolished':'DEST',
'Detention center/Jail/Prison':'JAIL',
'Diner/Restaurant':'REST',
'Dining hall/Cafeteria':'DINI',
'Electrical distribution':'ELEC',
'Dock/Pier/Wharf':'PIER',
'Drugstore/Pharmacy':'PHAR',
'Duplex':'DUPL',
'Educational-related':'EDUC',
'Electrical plant':'POWR',
'Electrical vault':'ELEC',
'Entertainment':'ENTE',
'Factory/Plant/Industrial':'PLNT',
'Farm':'FARM',
'Farmers market structure':'MARK',
'Farmstead':'FARM',
'Financial institution':'BANK',
'Firehouse':'FIRE',
'Fish house':'FISH',
'Fitness center/Spa/Gym':'GYMN',
'Fortification/Blockhouse':'FORT',
'Fraternal order building':'CLUB',
'Funeral home/Mortuary':'MORT',
'Garage':'GARA',
'Garage apartment':'GAPT',
'Gas station':'GAST',
'Gateway/Gatehouse/Arched entryway':'GATE',
'Gazebo':'GAZE',
'Golf course':'GOLF',
'Government Offices':'GOVT',
'Graveyard':'CEME',
'Greenhouse':'GREE',
'Grist/Lumber/Sugar mill':'MILL',
'Grocery store/Supermarket':'SGRO',
'Gym/Spa/Fitness center':'GYMN',
'Hairdresser':'SBEA',
'Hall':'HALL',
'Hangar':'HANG',
'Hardware store':'SHRD',
'Health center/Clinic/Outpatent care':'OUTP',
'Historic marker':'MONU',
'Home':'RESI',
'Hospital':'HOSP',
'Hotel':'HOTL',
'House':'RESI',
'House of worship':'HSEW',
'House, charnel':'CHAR',
'House of prostitution':'HPRO',
'Icehouse':'ICEH',
'Industrial':'PLNT',
'Industrial plant':'PLNT',
'Inn':'HOTL',
'Inpatient care':'HOSP',
'Jail/Prison/Detention Center':'JAIL',
'Kennel':'KENN',
'Kindergarten':'SCHO',
'Kitchen':'KITC',
'Labor/Migrant housing':'LABH',
'Laboratory':'SCIE',
'Launch pad':'LPAD',
'Laundry':'LAUN',
'Library':'LIBR',
'Lighthouse':'LIGH',
'Lock structure, canal':'CANL',
'Club or Lodge building':'CLUB',
'Lounge':'BAR',
'Lumber/Sugar/Grist mill':'MILL',
'Mall/Shopping center':'MALL',
'Marker/Monument/Statue':'MONU',
'Market structure':'MARK',
'Mausoleum':'MAUS',
'Medical offices':'OFFI',
'Medical, general':'MEDI',
'Meetinghouse, club':'CLUB',
'Meetinghouse, religious':'CHUR',
'Migrant/Labor housing':'LABH',
'Military':'MILI',
'Military base':'MILI',
'Mill':'MILL',
'Mobile Home/Trailer Home':'MOBH',
'Monastery/Convent':'CONV',
'Monument/Marker/Statue':'MONU',
'Moonshine still/Still/Distillery':'MOON',
'Mortuary/Funeral home':'MORT',
'Motel':'HOTL',
'Movie theater/Playhouse':'THEA',
'Multiple important uses':'MULT',
'Multipurpose public building':'CIVI',
'Municipal building':'CIHL',
'Museum/Gallery/Planetarium':'MUSE',
'Not applicable':'NA',
'Nursery school':'CHIL',
'Nursing home':'NURS',
'Office':'OFFI',
'Office building':'BOFF',
'Offices, dental':'OFFI',
'Offices, government':'GOVT',
'Offices, medical':'OFFI',
'Arena/Stadium':'STAD',
'Orphanage/Childen\'s home':'ORPH',
'Other':'OTHR',
'Outbuilding':'OUTB',
'Outhouse/Privy/Restroom':'OUTH',
'Outpatient care':'OUTP',
'Park':'PARK',
'Parsonage':'RESI',
'Pavilion':'PAVI',
'Penetentiary':'JAIL',
'Pharmacy/Drugstore':'PHAR',
'Physician\'s offie':'OFFI',
'Pier/Dock/Wharf':'PIER',
'Planetarium/Gallery/Museum':'MUSE',
'Plant, industrial':'PLNT',
'Plantation':'PLAN',
'Plaza/Square':'SQUA',
'Police station':'POLI',
'Pool, swimming':'POOL',
'Post office':'POST',
'Pound':'KENN',
'Power plant':'POWR',
'Preschool':'CHIL',
'Prison/Jail/Detention Center':'JAIL',
'Private Residence (House/Cottage/Cabin)':'RESI',
'Privy':'OUTH',
'Professional':'PROF',
'Professional office':'OFFI',
'Quadraplex':'APTM',
'Raceway':'RACE',
'Railroad depot':'TERM',
'Recreation hall':'BREC',
'Recreation, building':'BREC',
'Recreation, non-building':'RECN',
'Religious':'RELI',
'Research laboratory':'SCIE',
'Residence & commercial':'CMRE',
'Residence, private':'RESI',
'Resort complex/Theme park':'RESO',
'Restaurant':'REST',
'Restroom':'OUTH',
'Retail establishment':'STOR',
'Road':'ROAD',
'Rooming house':'ROOM',
'Ruin/Demolished/Destroyed':'DEST',
'Sailing vessel/Ship/Boat':'VESS',
'Salon/Barber/Beauty shop':'SBEA',
'Saloon':'BAR',
'Savings & loan':'BANK',
'Saw/Sugar/Grist mill':'MILL',
'School':'SCHO',
'Scientific laboratory':'SCIE',
'Seawall':'SEAW',
'Service station':'GAST',
'Sewage treatment/Waterworks':'WATW',
'Shed/Outbuilding':'OUTB',
'Ship/Sailing vessel':'VESS',
'Ship, sailing':'VESS',
'Shoe shop':'SHOE',
'Shop':'SHOP',
'Shop, auto repair':'GAST',
'Shop, bakery':'BAKE',
'Shop, beauty':'SBEA',
'Shop, blacksmith':'SBLK',
'Shop, shoe':'SHOE',
'Shopping center/Mall':'MALL',
'Slave quarters':'SLAV',
'Smithy':'SBLK',
'Smokehouse':'SMOK',
'Spa/Gym/Fitness center':'GYMN',
'Springhouse':'SPRG',
'Square/Plaza':'SQUA',
'Stable':'STAB',
'Stadium/Arena':'STAD',
'Statue/Monument/Marker':'MONU',
'Still/Distillery':'MOON',
'Storage building':'BSTO',
'Store':'STOR',
'Store, department':'SDEP',
'Store, grocery':'SGRO',
'Store, hardware':'SHRD',
'Sugar/Lumber/Grist mill':'MILL',
'Supermarket':'SGRO',
'Swimming pool':'POOL',
'Synagogue':'HSEW',
'Tavern':'BAR',
'Technical school':'SCHO',
'Temple (lodge)':'CLUB',
'Temple (religious)':'RELI',
'Temple, house of worship':'HSEW',
'Temple, social club':'CLUB',
'Terminal, Air/Bus/Rail':'TERM',
'Theater, movie or playhouse':'THEA',
'Theme park/Resort complex':'RESO',
'Tower':'TOWR',
'Tower, control':'CTOW',
'Town hall':'CIHL',
'Electrical transmission':'ELEC',
'Transportation':'TRAN',
'Triplex':'APTM',
'Turpentine Still':'TURP',
'University':'SCHO',
'Unknown':'UNKN',
'Unspecified':'UNSP',
'Unused':'VACA',
'Vacant':'VACA',
'Vault, electrical':'ELEC',
'Vehicle, road or rail':'VEHI',
'Vessel, watergoing':'VESS',
'Vocational school':'SCHO',
'Wall':'WALL',
'Warehouse':'BSTO',
'Water fountain':'WATF',
'Water supply structure/Water tower':'WSUP',
'Water tower/Water supply structure':'WSUP',
'Waterworks/Sewage treatment':'WATW',
'Wharf/Pier/Dock':'PIER',
'Women\'s club':'CLUB',
'Zoo':'ZOO',
'Abandoned':'VACA',
'Agricultural':'AGRI',
'Airport':'AIRP',
'Animal_shelter':'KENN',
'Apartment':'APTM',
'Apartment_garage':'GAPT',
'Apartments_commercial':'CMAP',
'Arched_entryway':'GATE',
'Auto_dealership':'AUTO',
'Auto_repair':'GAST',
'Bakery_shop':'BAKE',
'Band_shell':'BDSH',
'Bank':'BANK',
'Bar':'BAR',
'Barber_shop':'SBEA',
'Salon_beauty':'SBEA',
'Bed_Breakfast':'BEDB',
'Belltower':'TOWR',
'Clocktower':'TOWR',
'Bridge':'BRID',
'Bus_terminal':'TERM',
'Cemetery':'CEME',
'Church_Cemetery_Complex':'CHCE',
'Church':'HSEW',
'City_hall':'CIHL',
'Civic_center':'CIVI',
'Clinic_outpatient':'OUTP',
'Clubhouse':'CLUB',
'Commercial_residence':'CMRE',
'Commercial':'COMM',
'Communications_related':'CMUN',
'Convent':'CONV',
'Community_center':'BREC',
'Cottage':'RESI',
'Courthouse':'COUR',
'Day_care':'CHIL',
'Demolished':'DEST',
'Dentist':'OFFI',
'Department_store':'SDEP',
'Depot':'TERM',
'Detention center':'JAIL',
'Diner':'REST',
'Electrical_distribution':'ELEC',
'Dock':'PIER',
'Drugstore':'PHAR',
'Duplex':'DUPL',
'Educational_related':'EDUC',
'Electrical_plant':'POWR',
'Entertainment':'ENTE',
'Factory':'PLNT',
'Farm':'FARM',
'Farmers_market_structure':'MARK',
'Financial_institution':'BANK',
'Firehouse':'FIRE',
'Fitness_center':'GYMN',
'Fraternal order building':'CLUB',
'Funeral_home':'MORT',
'Garage':'GARA',
'Garage_apartment':'GAPT',
'Gas_station':'GAST',
'Gazebo':'GAZE',
'Golf_course':'GOLF',
'Government':'GOVT',
'Mill':'MILL',
'Grocery_store':'SGRO',
'Hall':'HALL',
'Hanger':'HANG',
'Hardware_store':'SHRD',
'Hospital':'HOSP',
'Hotel':'HOTL',
'House':'RESI',
'Industrial':'PLNT',
'Jail':'JAIL',
'Kennel':'KENN',
'Kindergarten':'SCHO',
'Laundry':'LAUN',
'Library':'LIBR',
'Lodge_club_building':'CLUB',
'Lounge':'BAR',
'Mall':'MALL',
'Medical_offices':'OFFI',
'Military':'MILI',
'Motel':'HOTL',
'Movie_theater':'THEA',
'Municipal_building':'CIHL',
'Museum_gallery_planetarium':'MUSE',
'Nursery_school':'CHIL',
'Nursing_home':'NURS',
'Office_building':'BOFF',
'Stadium':'STAD',
'Outbuilding':'OUTB',
'Park':'PARK',
'Parsonage':'RESI',
'Pavilion':'PAVI',
'Physician\'s_office':'OFFI',
'Pier':'PIER',
'Plaza':'SQUA',
'Police_station':'POLI',
'Pool_swimming':'POOL',
'Post_office':'POST',
'Power_plant':'POWR',
'Professional_office':'OFFI',
'Private_Residence':'RESI',
'Quadraplex':'APTM',
'Raceway':'RACE',
'Railroad_depot':'TERM',
'Recreation_non_building':'RECN',
'Religious':'RELI',
'Residence_private':'RESI',
'Restaurant':'REST',
'Retail_establishment':'STOR',
'School':'SCHO',
'Sewage_treatment':'WATW',
'Storage_building':'BSTO',
'Synagogue':'HSEW',
'Temple_religious':'RELI',
'Triplex':'APTM',
'University':'SCHO',
'Wall':'WALL',
'Warehouse':'BSTO',
'Women\'s_club':'CLUB',
'Zoo':'ZOO',
'Vacant':'VACA',
'Unknown':'UNKN',
}

#Replace Original Use text with codes:
input_all['OriginalUse'].replace(use_dict, inplace=True)
#Replace Current Use text with codes:
input_all['CurrentUse'].replace(use_dict, inplace=True)
#Replace Other Use text with codes:
input_all['OtherUse'].replace(use_dict, inplace=True)

In [ ]:
input_all['OrigUseFromYear'] = input_all['OrigUseFromYear'].replace('',0).astype(int).replace(0,'')
input_all['OrigUseToYear'] = input_all['OrigUseToYear'].replace('',0).astype(int).replace(0,'')
input_all['CurrUseFromYear'] = input_all['CurrUseFromYear'].replace('',0).astype(int).replace(0,'')
input_all['CurrUseToYear'] = input_all['CurrUseToYear'].replace('',0).astype(int).replace(0,'')
input_all['OtherUseFromYear'] = input_all['OtherUseFromYear'].replace('',0).astype(int).replace(0,'')
input_all['OtherUseToYear'] = input_all['OtherUseToYear'].replace('',0).astype(int).replace(0,'')


In [ ]:
display(input_all)

In [ ]:
#Lookup table: Style

style_dict = {
   'British Colonial':'BRCO',
'Brutalism':'BRUT',
'Art Deco':'ARTD',
'Bungalow':'BUNG',
'Byzantine Revival':'MOOR',
'Chateauesque':'FRCO',
'Classical Revival':'NEOR',
'Colonial Revival':'GEOR',
'Commercial':'COMM',
'Egyptian Revival':'EGYR',
'Frame Vernacular':'FRAM',
'Second Empire':'FR2E',
'Georgian Revival':'GEOR',
'Masonry Vernacular':'MASO',
'Gothic Revival':'GOTR',
'Greek Revival':'GRKR',
'International':'INTE',
'Sarasota School':'SARA',
'Italianate':'ITAL',
'Log':'LOG',
'Mediterranean Revival':'MEDR',
'Mission':'MISS',
'Moderne':'MODE',
'Modernistic':'MODE',
'Mid-Century Modern':'MIDM',
'Neo-Classical Revival':'NEOR',
'Prairie':'PRAI',
'Queen Anne (Revival)':'QUEE',
'Italian Renaissance Revival':'ITRE',
'Richardsonian Romanesque':'ROMR',
'Romanesque (Revival)':'ROMR',
'Shingle':'SHNG',
'Spanish Colonial':'SPCO',
'Stick':'STIC',
'Tudor Revival':'TUDR',
'Other':'OTHR',
'Gothic-Collegiate':'GOTC',
'Not applicable':'NA',
'Industrial Vernacular':'INDU',
'American Foursquare':'BOX',
'Craftsman':'CRAF',
'Unspecified':'UNSP',
'Minimal Traditional':'MTRA',
'Ranch':'RANC',
'Victorian (Italianate)':'ITAL',
'Mansard':'FR2E',
'Jacobean Revival':'TUDR',
'Elizabethan Revival':'TUDR',
'Beaux Arts Eclecticism':'BEAU',
'Pueblo':'PUEB',
'Spanish Revival':'MEDR',
'French Renaissance':'FRRE',
'Meisian':'INTE',
'Bauhausian':'INTE',
'English Tudor Cottage':'TUDC',
'Eng Cotswold Cottage':'COTC',
'Eng Georgian Cottage':'GEOC',
'New England Cottage':'NENC',
'French Cottage':'FREC',
'Spanish Cottage':'SPNC',
'Dutch Cottage':'DUTC',
'Box (Amer 4 Square)':'BOX',
'Split Level':'SPLV',
'French Colonial':'FRCO',
'Octagon':'OCTA',
'Moorish Revival':'MOOR',
'Collegiate Gothic':'GOTC',
'Monterey':'MONT',
'Art Moderne':'MODE',
'Streamlined Moderne':'MODE',
'Spanish Eclectic':'MEDR',
'Dutch Colonial Revival':'DUTC',
'Span Colonial Revival':'MEDR',
'Georgian':'BRCO',
'No style':'NOST',
'Mixed, none dominant':'MIXD',
'Renaissance Revival':'ITRE',
'Federal (Adamesque)':'FEDE',
'Adams/Adamesque':'FEDE',
'Adamesque Revival':'NEOR',
'Federal Revival':'NEOR',
'Folk Victorian, Frame':'FRAM',
'Folk Victorian, Masonry':'MASO',
'International':'INTE',
'Mid_Century_Modern':'MIDM',
'Commercial':'COMM',
'Frame_Vernacular':'FRAM',
'Masonry_Vernacular':'MASO',
'American_Foursquare':'BOX',
'Art_Deco':'ARTD',
'Beaux_Arts_Eclecticism':'BEAU',
'Brutalism':'BRUT',
'Bungalow':'BUNG',
'Byzantine_Revival':'MOOR',
'Chateauesque':'FRCO',
'Classical_Revival':'NEOR',
'Colonial_Revival':'GEOR',
'Craftsman':'CRAF',
'Egyptian_Revival':'EGYR',
'Folk_Victorian_Frame':'FRAM',
'Folk_Victorian_Masonry':'MASO',
'French_Colonial':'FRCO',
'French_Renaissance':'FRRE',
'Georgian_Revival':'GEOR',
'Gothic_Revival':'GOTR',
'Gothic_Collegiate':'GOTC',
'Greek_Revival':'GRKR',
'Industrial_Vernacular':'INDU',
'Italian_Renaissance_Revival':'ITRE',
'Italianate':'ITAL',
'Log':'LOG',
'Mediterranean_Revival':'MEDR',
'Minimal_Traditional':'MTRA',
'Mission':'MISS',
'Moderne':'MODE',
'Monterey':'MONT',
'Moorish_Revival':'MOOR',
'Neo_Classical_Revival':'NEOR',
'Octagon':'OCTA',
'Prairie':'PRAI',
'Pueblo':'PUEB',
'Queen_Anne_Revival':'QUEE',
'Ranch':'RANC',
'Renaissance_Revival':'ITRE',
'Richardsonian_Romanesque':'ROMR',
'Romanesque_Revival':'ROMR',
'Sarasota_School':'SARA',
'Second_Empire':'FR2E',
'Shingle':'SHNG',
'Spanish_Colonial':'SPCO',
'Spanish_Eclectic':'MEDR',
'Spanish_Revival':'MEDR',
'Split_Level':'SPLV',
'Stick':'STIC',
'Streamline_Moderne':'MODE',
'Tudor_Revival':'TUDR',
'Mixed_none_dominant':'NOST',
'Not_applicable':'NA',
'Other':'OTHR',
'Mid_Century_Modern':'MIDM'
}

#Replace Style text with codes:
input_all['Style'].replace(style_dict, inplace=True)


In [ ]:
#Lookup table: Exterior Plan

exterior_dict = {
    'Central Block Symmetric Wings':'BLKW',
'Central Block, 1 wing':'BLK1',
'Greek cross':'GRCR',
'Irregular':'IRRE',
'L-shaped':'LSHP',
'Latin cross':'LACR',
'Rectangular':'RECT',
'Square':'SQAR',
'T-shaped':'TSHP',
'U-shaped':'USHP',
'Other':'OTHR',
'E-shaped':'ESHP',
'I-shaped':'ISHP',
'Not applicable':'NA',
'Unspecified':'UNSP',
'Octagonal':'OCTA',
'Circular':'CIRC',
'Irregular':'IRRE',
'L_shaped':'LSHP',
'Rectangular':'RECT',
'Square':'SQAR',
'T_shaped':'TSHP',
'U_shaped':'USHP',
'Latin_cross':'LACR',
'Greek_Cross':'GRCR',
'Central_Block_1_wing':'BLK1',
'Central_Block_Symmetric_Wings':'BLKW',
'Not_applicable':'NA',
}


#Replace Exterior Plan text with codes:
input_all['ExteriorPlan'].replace(exterior_dict, inplace=True)

In [ ]:
#Lookup table: Exterior Fabric

extfab_dict = {
    'Aluminum':'ALUM',
'Artbrick, artstone':'PEDA',
'Artif masonry veneer':'AMAV',
'Asbestos':'SHAS',
'Asphalt-rolled':'ROAS',
'Beaded weatherboard':'WTBD',
'BEC ceiling board':'BECC',
'Block-concrete':'COBL',
'Block-concrete-mold':'COBM',
'Block-coquina':'COQB',
'Board and batten':'BDBT',
'Brick':'BRIC',
'Brick-tabby':'TABB',
'Carrara glass':'GLPS',
'Cast iron':'CAIR',
'Caststone':'AMAV',
'Cement-mineral fiber':'MFCE',
'Clapboard':'WTBD',
'Concrete':'CONC',
'Concrete block':'COBL',
'Concrete block-mold':'COBM',
'Concrete-poured':'COPO',
'Concrete-pre-cast':'COPC',
'Coquina block':'COQB',
'Drop siding':'SDDR',
'Fiberglass':'GLFI',
'Flush wood siding':'SDFW',
'Glass block':'GLBL',
'Glass-fiberglass':'GLFI',
'Glass-pigmentd structural':'GLPS',
'Glazed masonry':'MAGL',
'Hewn log':'LGHW',
'Horizontal plank':'PLHO',
'Iron-cast':'CAIR',
'Log-unspecified':'LOG',
'Log-hewn':'LGHW',
'Log-round':'LGRO',
'Log-split':'LGSP',
'Masonry veneer-artificial':'AMAV',
'Masonry-glazed':'MAGL',
'Metal':'META',
'Metal-pressed':'PRME',
'Mineral fiber cement':'MFCE',
'Molded concrete block':'COBM',
'Narrow routed siding':'SDNR',
'Novelty siding':'SDDR',
'Pebble dash':'PEDA',
'Pigmented structural glass':'GLPS',
'Plank-horizontal':'PLHO',
'Plank-vertical':'PLVE',
'Plastic':'PLAS',
'Poured concrete':'COPO',
'Poured tabby':'TABP',
'Pre-cast concrete':'COPC',
'Pressed metal':'PRME',
'Pressed stone':'COBM',
'Rolled asphalt':'ROAS',
'Round log':'LGRO',
'Shingles-asbestos':'SHAS',
'Shingles-wood':'SHWD',
'Shiplap':'SHIP',
'Siding-drop':'SDDR',
'Siding-narrow routed':'SDNR',
'Siding-novelty':'SDDR',
'Split log':'LGSP',
'Steel':'STEE',
'Stone':'STON',
'Structural glass-pigmented':'GLPS',
'Stucco':'STUC',
'Tabby brick':'TABB',
'Tabby-poured':'TABP',
'Terra cotta':'TECO',
'Vertical plank':'PLVE',
'Vinyl':'VINY',
'Weatherboard':'WTBD',
'Weatherboard-beaded':'WTBD',
'Window wall':'WINW',
'Wood shingles':'SHWD',
'Wood siding':'SDW',
'Wood/Plywood':'WOOD',
'Other':'OTHR',
'Unknown':'UNKN',
'Shingles-unspecified':'SHIN',
'Tar paper':'TARP',
'Not applicable':'NA',
'Shingles-slate':'SHSL',
'Slate shingles':'SHSL',
'Unspecified':'UNSP',
'Asphalt shingles':'SHIN',
'Shingles-asphalt':'SHIN',
'Composition Board':'COBD',
'Composition Roll':'ROAS',
'Random Ashlar':'STON',
'Stucco':'STUC',
'Brick':'BRIC',
'Block_concrete':'COBL',
'Terra_Cotta':'TECO',
'Masonry_Glazed':'MAGL',
'Aluminum':'ALUM',
'Asbestos':'SHAS',
'Shingles_asbestos':'SHAS',
'Shingles_wood':'SHWD',
'Shingles_slate':'SHSL',
'Wood_siding':'SDW',
'Siding_drop':'SDDR',
'Siding_narrow_routed':'SDNR',
'Siding_novelty':'SDDR',
'Shiplap':'SHIP',
'Block_concrete_molded':'COBM',
'Concrete_poured':'COPO',
'Concrete_precast':'COPC',
'Coquina_block':'COQB',
'Masonry_veneer_artificial':'AMAV',
'Carrara_glass':'GLPS',
'Beaded_weatherboard':'WTBD',
'Cast_iron':'CAIR',
'Log':'LOG',
'Fiberglass':'GLFI',
'Glass_block':'GLBL',
'Vinyl':'VINY',
'Stone':'STON',
'Caststone':'AMAV',
'Window_wall':'WINW',
'Metal':'META',
'Metal_pressed':'PRME',
'Mineral_fiber_cement':'MFCE',
'other':'OTHR',
'unknown':'UNKN',
'not applicable':'NA',

}


#Replace Exterior Fabric text with codes:
input_all['ExteriorFabric1'].replace(extfab_dict, inplace=True)
input_all['ExteriorFabric2'].replace(extfab_dict, inplace=True)
input_all['ExteriorFabric3'].replace(extfab_dict, inplace=True)

In [ ]:
#Lookup table: Roof Type

rooftype_dict = {
    'Built-up':'FLAT',
'Clipped gable':'JERK',
'Cone':'CONE',
'Dome':'DOME',
'Flat':'FLAT',
'Gable':'GA',
'Gable on hip':'GAHP',
'Gable-stepped':'GAST',
'Gable-intersecting':'GAIN',
'Gambrel':'GAMB',
'Hip':'HIP',
'Hip on hip':'HPHP',
'Intersecting gables':'GAIN',
'Jerkin head':'JERK',
'Mansard':'MANS',
'Monitor':'MONI',
'Pyramid':'PYRA',
'Shed':'SHED',
'Stepped Gable':'GAST',
'Vaulted':'VAUL',
'Other':'OTHR',
'Not applicable':'NA',
'Unspecified':'UNSP',
'Cross-gabled':'GAIN',
'Bowed-arched':'BOWD',
'Gable':'GA',
'Gable_on_hip':'GAHP',
'Clipped_gable':'JERK',
'Jerkin_head':'JERK',
'Intersecting_gables':'GAIN',
'Cross_gabled':'GAIN',
'Stepped_gable':'GAST',
'Flat':'FLAT',
'Hip':'HIP',
'Hip_on_hip':'HPHP',
'Mansard':'MANS',
'Shed':'SHED',
'Gambrel':'GAMB',
'Dome':'DOME',
'Pyramid':'PYRA',
'Other':'OTHR',
'Not_applicable':'NA',
}

#Replace Roof Type text with codes:
input_all['RoofType1'].replace(rooftype_dict, inplace=True)
input_all['RoofType2'].replace(rooftype_dict, inplace=True)
input_all['RoofType3'].replace(rooftype_dict, inplace=True)

In [ ]:
#Lookup table: Roof Material

roofmat_dict = {
    'Barrel tile':'TIMI',
'Built-up':'BUIL',
'Composition roll':'CROL',
'Composition shingles':'SHCO',
'Flat tile':'TIFL',
'French tile':'TIFL',
'Metal shingles':'SHME',
'Mission tile':'TIMI',
'Pantile':'TIPA',
'Sheet metal:3V crimp':'SM3V',
'Sheet metal:5V crimp':'SM5V',
'Sheet metal:corrugated':'SMCO',
'Sheet metal:standing seam':'SMSS',
'Slate shingles':'SHSL',
'Spanish tile':'TISP',
'Tile unspecified':'TILE',
'Wood shingles':'SHWD',
'Other':'OTHR',
'Tar & gravel':'T&G',
'Concrete tile':'TICO',
'Unspecified':'UNSP',
'Tin':'TIN',
'Shingle, unspecified':'SHIN',
'Asphalt shingles':'ASPH',
'Built_up':'BUIL',
'Asphalt_shingles':'ASPH',
'Barrel_tile':'TIMI',
'Tile_unspecified':'TILE',
'Sheet_metal_standing_ seam':'SMSS',
'Sheet_metal_3V_crimp':'SM3V',
'Sheet_metal_5V_crimp':'SM5V',
'Metal_shingles':'SHME',
'Tar_and_gravel':'T&G',
'Tin':'TIN',
'Other':'OTHR',
'Unspecified':'UNSP',
}

#Replace Roof Material text with codes:
input_all['RoofMaterial1'].replace(roofmat_dict, inplace=True)
input_all['RoofMaterial2'].replace(roofmat_dict, inplace=True)
input_all['RoofMaterial3'].replace(roofmat_dict, inplace=True)

In [ ]:
#Lookup table: Roof Secondary Structures

secstr_dict = {
    'Bellcote':'BELC',
'Belvedere':'BELV',
'Cone tower':'TWCO',
'Cone turret':'TUCO',
'Cross gable':'CRGA',
'Cupola':'CUPO',
'Dome':'DOME',
'Eyebrow dormer':'DREY',
'Flat dormer':'DRFL',
'Gable dormer':'DRGA',
'Hip dormer':'DRHI',
'Pavilion tower':'TWPA',
'Pavilion turret':'TUPA',
'Scuttle':'SCUT',
'Shed dormer':'DRSH',
'Steeple':'STEE',
'Other':'OTHR',
'Not applicable':'NA',
'Unspecified':'UNSP',
'Gable extension':'EXGA',
'Hip extension':'EXHI',
'Shed extension':'EXSH',
'Flat extension':'EXFL',
'Eyebrow_dormer':'DREY',
'Flat_dormer':'DRFL',
'Gable_dormer':'DRGA',
'Hip_dormer':'DRHI',
'Shed_dormer':'DRSH',
'Belvedere':'BELV',
'Cupola':'CUPO',
'Dome':'DOME',
'Pavilion_tower':'TWPA',
'Cone_turret':'TUCO',
'Steeple':'STEE',
'Gable_extension':'EXGA',
'Hip_extension':'EXHI',
'Shed_extension':'EXSH',
'Flat_extension':'EXFL',
}


#Replace Roof Secondary Structures text with codes:
input_all['RoofSecondaryStruc1'].replace(secstr_dict, inplace=True)
input_all['RoofSecondaryStruc2'].replace(secstr_dict, inplace=True)

In [ ]:
#Lookup table: Chimney Materials

chimat_dict = {
    'Brick':'BRIC',
'Concrete':'CON',
'Concrete block':'CONB',
'Metal':'METL',
'Mud':'MUD',
'Stucco':'STUC',
'Stone':'STON',
'Masonry':'MASO',
'Tile':'TILE',
'Wood':'WOOD',
'Other':'OTHR',
'Not applicable':'NA',
'Unspecified':'UNSP',
'Brick':'BRIC',
'Concrete_block':'CONB',
'Concrete':'CON',
'Metal':'METL',
'Tile':'TILE',
'Wood':'WOOD',
'Stucco':'STUC',
'Stone':'STON',
'Other':'OTHR',  
}

#Replace Chimney Materials text with codes:
input_all['ChimneyMaterials1'].replace(chimat_dict, inplace=True)
input_all['ChimneyMaterials2'].replace(chimat_dict, inplace=True)

In [ ]:
#Lookup table: Structural Systems

strsys_dict = {
    'Balloon wood frame':'WFBL',
'Braced wood frame':'WFBR',
'Brick':'BRIC',
'Butt joint log':'LGBJ',
'Cast iron skeleton':'SKCI',
'Cast-in-place concrete':'CNCP',
'Concrete':'CONC',
'Concrete block':'CONB',
'Coquina block':'COQB',
'Dovetail log':'LGDV',
'Glued-lam wood/plywood':'GLUL',
'Glued-laminated':'GLUL',
'Heavy timber':'WFBR',
'Hollow clay tile':'STCT',
'Laminated Wood':'GLUL',
'Lapped log':'LGLP',
'Light wood frame':'WFLT',
'Log':'LG',
'Masonry - General':'MASO',
'Metal skeleton':'SKME',
'Plank wall':'PLKW',
'Platform wood frame':'WFPL',
'Post and beam':'WFBR',
'Pre-cast concrete':'CNPC',
'Reinforced concrete':'CNRE',
'Retaining wall':'RETW',
'Saddle notched log':'LGSN',
'Skeleton-cast iron':'SKCI',
'Skeleton-metal':'SKME',
'Skeleton-steel':'SKST',
'Skeleton-wrought iron':'SKWI',
'Space frame':'SPFR',
'Steel skeleton':'SKST',
'Stone':'STON',
'Structural clay tile':'STCT',
'Tabby poured':'TABB',
'Tensile':'TENS',
'Tongue & groove log':'LGTG',
'V-notched log':'LGVN',
'Wood frame':'WF',
'Wrought iron skeleton':'SKWI',
'Other':'OTHR',
'Unknown':'UNKN',
'Not applicable':'NA',
'Unspecified':'UNSP',
'Balloon_wood_frame':'WFBL',
'Braced_wood_frame':'WFBR',
'Brick':'BRIC',
'Butt_joint_log':'LGBJ',
'Cast_iron_skeleton':'SKCI',
'Cast_in_place_concrete':'CNCP',
'Concrete_block':'CONB',
'Coquina_block':'COQB',
'Hollow_clay_tile':'STCT',
'Light_wood_frame':'WFLT',
'Log':'LG',
'Masonry_general':'MASO',
'Metal_skeleton':'SKME',
'Post_and_beam':'WFBR',
'Pre_cast_concrete':'CNPC',
'Reinforced_concrete':'CNRE',
'Saddle_notched_log':'LGSN',
'Steel_skeleton':'SKST',
'Stone':'STON',
'Structural_clay_tile':'STCT',
'Wood_frame':'WF',
'Other':'OTHR',
'Unknown':'UNKN',
}

#Replace Structural Systems text with codes:
input_all['StrucSys1'].replace(strsys_dict, inplace=True)
input_all['StrucSys2'].replace(strsys_dict, inplace=True)
if 'StrucSys3' in input_all.columns: input_all['StrucSys3'].replace(strsys_dict, inplace=True)

In [ ]:
#Lookup table: Foundation Types

fndtyp_dict = {
    'Continuous':'CONT',
'Piers':'PIER',
'Slab':'SLAB',
'Other':'OTHR',
'Unspecified by recorder':'UNSP',
'Unknown':'UNKN',
'Continuous':'CONT',
'Piers':'PIER',
'Slab':'SLAB',
'Other':'OTHR',
'Unspecified_by_recorder':'UNSP',
'Unknown':'UNKN',
}

#Replace Foundation Types text with codes:
input_all['FoundationType1'].replace(fndtyp_dict, inplace=True)
input_all['FoundationType2'].replace(fndtyp_dict, inplace=True)

In [ ]:
#Lookup table: Foundation Materials

fndmat_dict = {
    'Brick':'BRIC',
'Concrete Block':'BLCO',
'Poured Concrete Footing':'POCO',
'Pre-cast Concrete Footing':'PREC',
'Concrete, Generic':'CONC',
'Clay Tile':'CLTI',
'Floating':'FLOA',
'Pile':'PILE',
'Stone':'STON',
'Tabby':'TABB',
'Wood Blocks':'BLWD',
'Other':'OTHR',
'Unspecified':'UNSP',
'Obscured':'OBSC',
'Brick':'BRIC',
'Concrete_Block':'BLCO',
'Poured_Concrete_Footing':'POCO',
'Pre_cast_Concrete_Footing':'PREC',
'Concrete_Generic':'CONC',
'Clay_Tile':'CLTI',
'Floating':'FLOA',
'Pile':'PILE',
'Stone':'STON',
'Tabby ':'TABB',
'Wood_Blocks':'BLWD',
'Other':'OTHR',
'Obscured':'OBSC',
}

#Replace Foundation Materials text with codes:
input_all['FoundationMatl1'].replace(fndmat_dict, inplace=True)
if 'FoundationMatl2' in input_all.columns:
   input_all['FoundationMatl2'].replace(fndmat_dict, inplace=True)

In [ ]:
display(input_all['Condition'])

In [ ]:
#Lookup table: Condition

cond_dict = {
    'excellent':'EXCE',
'good':'GOOD',
'fair':'FAIR',
'deteriorated':'DETE',
'ruinous':'RUIN',
'Excellent':'EXCE',
'Good':'GOOD',
'Fair':'FAIR',
'Deteriorated':'DETE',
'Ruinous':'RUIN',
}

#Replace Condition text with codes:
input_all['Condition'].replace(cond_dict, inplace=True)

In [ ]:
# Initialize Research Methods variables:

input_all['ResMethFMSF'] = 'Off'
input_all['ResMethARCHIVES'] = 'Off'
input_all['ResMethAPPRAISER'] = 'Off'
input_all['ResMethSURVEY'] = 'Off'
input_all['ResMethLIBRARY'] = 'Off'
input_all['ResMethPERMITS'] = 'Off'
input_all['ResMethSANBORN'] = 'Off'
input_all['ResMethCITY'] = 'Off'
input_all['ResMethOCCINTVIEW'] = 'Off'
input_all['ResMethPLAT'] = 'Off'
input_all['ResMethNEWSPAPER'] = 'Off'
input_all['ResMethNEIGINTVIEW'] = 'Off'
input_all['ResMethPLSS'] = 'Off'
input_all['ResMethPHOTOS'] = 'Off'
input_all['ResMethINSPECTION'] = 'Off'
input_all['ResMethHABS'] = 'Off'
input_all['ResMethOTHR'] = 'Off'

In [ ]:
# Research Methods Codes:


#Replace Research Methods selection with codes:

input_all.loc[input_all['ResMethods'].apply(lambda x: 'propert_appraiser_tax_records' in x), 'ResMethAPPRAISER'] = 'APPR'
input_all.loc[input_all['ResMethods'].apply(lambda x: 'FL_State_Archive' in x), 'ResMethARCHIVES'] = 'FLAR'
input_all.loc[input_all['ResMethods'].apply(lambda x: 'cultural_resource_survey' in x), 'ResMethSURVEY'] = 'CRAS'
input_all.loc[input_all['ResMethods'].apply(lambda x: 'city_directory' in x), 'ResMethCITY'] = 'CITY'
input_all.loc[input_all['ResMethods'].apply(lambda x: 'newspaper_files' in x), 'ResMethNEWSPAPER'] = 'NEWS'
input_all.loc[input_all['ResMethods'].apply(lambda x: 'historic_photos' in x), 'ResMethPHOTOS'] = 'PHOT'
input_all.loc[input_all['ResMethods'].apply(lambda x: 'building_permits' in x), 'ResMethPERMITS'] = 'BLDG'
input_all.loc[input_all['ResMethods'].apply(lambda x: 'other' in x), 'ResMethOTHR'] = 'OTHR'
input_all.loc[input_all['ResMethods'].apply(lambda x: 'Sanborn_maps' in x), 'ResMethSANBORN'] = 'SANB'
input_all.loc[input_all['ResMethods'].apply(lambda x: 'Public_Lands_Survey' in x), 'ResMethPLSS'] = 'PUBL'
input_all.loc[input_all['ResMethods'].apply(lambda x: 'FMSF_record_search' in x), 'ResMethFMSF'] = 'FMSF'
input_all.loc[input_all['ResMethods'].apply(lambda x: 'neighbor_interview' in x), 'ResMethNEIGINTVIEW'] = 'NVNR'
input_all.loc[input_all['ResMethods'].apply(lambda x: 'occupant_owner_interview' in x), 'ResMethOCCINTVIEW'] = 'NVCR'
input_all.loc[input_all['ResMethods'].apply(lambda x: 'interior_inspection' in x), 'ResMethINSPECTION'] = 'INTR'
input_all.loc[input_all['ResMethods'].apply(lambda x: 'plat_maps' in x), 'ResMethPLAT'] = 'PLAT'
input_all.loc[input_all['ResMethods'].apply(lambda x: 'library_research' in x), 'ResMethLIBRARY'] = 'LIBR'
input_all.loc[input_all['ResMethods'].apply(lambda x: 'HABS_HAER_record_search' in x), 'ResMethHABS'] = 'HABS'



In [ ]:
display(input_all)

In [ ]:
#Lookup Table: Areas of Historical Significance

hisass_dict = {
    'Aboriginal':'ABOR',
'African American':'BLAC',
'Agriculture':'AGRI',
'Archaeology-historic':'ARCH',
'Archaeology-prehistoric':'ARCP',
'Architecture':'ARCT',
'Art':'ART',
'Athletics':'ATHL',
'Black history':'BLAC',
'Commerce':'COMM',
'Communications':'CMCN',
'Community planning & development':'CPDV',
'Conservation':'CONS',
'Cuban heritage':'CUBA',
'Economics':'ECON',
'Education':'EDUC',
'Engineering':'ENGI',
'Entertainment/recreation':'ENRE',
'Ethnic heritage':'ETHN',
'Exploration/settlement':'EXPL',
'Health/medicine':'HLTH',
'Industry':'INDU',
'Invention':'INVE',
'Jewish heritage':'JEWH',
'Landscape architecture':'LAAR',
'Law':'LAW',
'Literature':'LITE',
'Local':'LOCA',
'Maritime history':'MARI',
'Military':'MILI',
'Music':'MUSI',
'Other':'OTHR',
'Performing arts/theater':'PATH',
'Philosophy':'PHIL',
'Politics/government':'POGO',
'Religion':'RELI',
'Science':'SCIE',
'Sculpture':'SCUL',
'Social/humanitarian':'SOHU',
'Tourism':'TOUR',
'Transportation':'TRAN',
'Unspecified on form':'UNSP',
'Agriculture':'AGRI',
'Architecture':'ARCT',
'Black_history':'BLAC',
'Commerce':'COMM',
'Community_planning_development':'CPDV',
'Ethnic_heritage':'ETHN',
'Education':'EDUC',
'Engineering':'ENGI',
'Entertainment_recreation':'ENRE',
'Exploration_settlement':'EXPL',
'Local':'LOCA',
'Politics_government':'POGO',
'Religion':'RELI',
'Transportation':'TRAN',
'Tourism':'TOUR',
'Industry':'INDU',
'Landscape_architecture':'LAAR',
'Maritime_history':'MARI',
'Military':'MILI',
'Social_humanitarian':'SOHU',
}


#Replace Areas of Significance text with codes:
input_all['HistAssociation1'].replace(hisass_dict, inplace=True)
input_all['HistAssociation2'].replace(hisass_dict, inplace=True)
input_all['HistAssociation3'].replace(hisass_dict, inplace=True)
input_all['HistAssociation4'].replace(hisass_dict, inplace=True)
input_all['HistAssociation5'].replace(hisass_dict, inplace=True)
input_all['HistAssociation6'].replace(hisass_dict, inplace=True)


In [ ]:
#Lookup Table: Document Type

repcat_dict = {
    'All materials at one location':'ALL',
'Artifact collection-closed to public':'ARTC',
'Artifact collection-open to public':'ARTO',
'Photographs':'PHOT',
'Field notes':'NOTE',
'Field maps':'MAP',
'Other collection / sample':'OTHR',
'None known':'NONE',
'Unspecified by recorder':'UNSP',
'All_materials_one_location':'ALL',
'Artifact_collection_closed':'ARTC',
'Artifact_collectio_open':'ARTO',
'Photographs':'PHOT',
'Field_notes':'NOTE',
'Field_maps':'MAP',
'Other_collection_sample':'OTHR',
'None_known':'NONE',
'Unspecified_by_recorder':'UNSP',
}

#Replace Areas of Significance text with codes:
input_all['RepositoryCat1'].replace(repcat_dict, inplace=True)
if 'RepositoryCat2' in input_all.columns: input_all['RepositoryCat2'].replace(repcat_dict, inplace=True)

In [ ]:
#Lookup Table: Maintaining Organization

reporg_dict = {
    'University of Florida':'UF',
    'Other':'OTHR',
}

#Replace Areas of Significance text with codes:
input_all['RepositoryOrg1'].replace(reporg_dict, inplace=True)
if 'RepositoryOrg2' in input_all.columns: input_all['RepositoryOrg2'].replace(reporg_dict, inplace=True)

In [ ]:
#Lookup Table: Affiliation 

input_all['RecorderAffiliation']

In [ ]:
# Force numerical values to integers:
numcols = ['Block','YrBuilt','Stories','ChimneyNo', 'Section1' ]

for col in numcols:
    numeric_mask = pd.to_numeric(input_all[col], errors='coerce').notnull()
    input_all.loc[numeric_mask, col] = input_all.loc[numeric_mask, col].astype(int)

In [ ]:
test_dict = input_all.iloc[0].to_dict()

In [ ]:
test_dict

In [ ]:
# Insert an initial row to fit in the form properly:

# List of fields to apply the transformation
fields_to_transform = ['OwnershipHistory','Windows', 'DistinguishingFeatures','AncillaryFeatures','MainEntrance',
       'PorchDescriptions','NarrativeDescription','BibliographicRef','ExplanationEvaluation']

# Function to add an extra carriage return at the beginning of each string
def add_extra_carriage_return(text):
    return '\n' + text

# Apply the function to each field in the DataFrame
for field in fields_to_transform:
    input_all[field] = input_all[field].apply(lambda x: add_extra_carriage_return(x))


In [ ]:
# Handle fields that are longer than their character limit:


char_len_dict = {
    'OwnershipHistory':150,
    'Windows':150,
    'DistinguishingFeatures':150 ,
    'AncillaryFeatures':150 ,
    'MainEntrance':120,
    'PorchDescriptions':254 ,
    'NarrativeDescription':254 ,
    'BibliographicRef':254 ,
    'ExplanationEvaluation':254 ,
}


input_all_cont = input_all[['SiteID','OwnershipHistory','Windows', 'DistinguishingFeatures','AncillaryFeatures','MainEntrance',
       'PorchDescriptions','NarrativeDescription','BibliographicRef','ExplanationEvaluation']].copy()



for col, limit in char_len_dict.items():
    input_all_cont[col] = input_all_cont[col].apply(lambda x: pd.NA if len(str(x)) < limit else x)

# Remove rows where all columns are blank

input_all_cont.dropna(subset=input_all_cont.columns.difference(['SiteID']), how='all', inplace=True)

# Replace too-long values in input_all with 'See Continuation Sheet'
for col, limit in char_len_dict.items():
    input_all[col] = input_all[col].apply(lambda x: '\n See Continuation Sheet' if len(str(x)) >= limit else x)



In [ ]:
# Rename column names for Continuation Sheet:

new_cont_columns = {
   'SiteID':'Site Number', 'OwnershipHistory':'Ownership History',
       'DistinguishingFeatures':'Distinguishing Features', 'AncillaryFeatures':'Ancillary Features', 'MainEntrance':'Main Entrance',
       'PorchDescriptions':'Porch Descriptions', 'NarrativeDescription':'Narrative Description', 'BibliographicRef':'Bibliographic References',
       'ExplanationEvaluation':'Explanation of Evaluation',
}

input_all_cont.rename(columns=new_cont_columns, inplace=True)

In [ ]:
display(input_all_cont)

In [ ]:
# Combine continuation sheet fields into a string:

def combine_fields(row, column_names):
    # Combine column names with corresponding values and filter out null values
    combined_values = [f"{column}: {value}" for column, value in zip(column_names, row) if pd.notnull(value)]
    # Join the combined values into a long string
    combined_string = ' <br/> <br/>'.join(combined_values)
    return combined_string

# Get the column names from the input_all DataFrame
column_names = input_all_cont.columns

# Apply the function to each row of the DataFrame
input_all_cont['combined_string'] = input_all_cont.apply(combine_fields, args=(column_names,), axis=1)
input_all_cont.reset_index(inplace= True)
input_all_cont.columns

In [ ]:
# This function generates the continuation sheet as a PDF: 

def generate_pdf(long_string, filename):
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter

    # Define the font and size
    font_name = "Courier"
    font_size = 10

    # Wrap the long string
    lines = []
    #line = ""
    #for word in long_string.split():
    #    if word != '<br/>' and c.stringWidth(line + " " + word, font_name, font_size) < width - 2*inch:
    #        line += " " + word
    #        print(c.stringWidth(line + " " + word, font_name, font_size))
    #        print(line)
    #    else:
    #        lines.append(line)
    #        line = word 
    
    #lines.append(line)
    spaces = "&nbsp;" * 40
    lines = ['Continuation Sheet' + spaces + long_string]
    # Justify the text
    styles = getSampleStyleSheet()
    style = styles["Normal"]
    style.fontName = font_name  # Set the font to Courier
    style.fontSize = font_size  # Set the font size
    style.alignment = 4  # Justify
    p = Paragraph("<br />".join(lines), style)
    p.wrapOn(c, width - 2*inch, height - 2*inch)

    # Calculate the vertical position for the text
    text_height = p.height
    vertical_position = height - 1*inch - text_height

    # Draw the text on the canvas at the calculated position
    p.drawOn(c, inch, vertical_position)

    c.save()




In [ ]:
# Example usage
# long_string = input_all_cont['combined_string'][1]
# pdf_filename = input_all_cont['Site Number'][1]+'_Continuation_Sheet.pdf'
# generate_pdf(long_string, pdf_filename)

In [ ]:
# Generate forms: # this step requires having Form_Structure_v50.pdf in the folder 
for row in range(len(input_all)):
    test_dict = input_all.iloc[row].to_dict()
    pypdftk.fill_form('Form_Structure_v50.pdf',test_dict, f"{test_dict['SiteID']}_Form_FMS.pdf", flatten= False)

In [ ]:
# Generate continuation sheets:

for row in range(len(input_all_cont)):
    long_string = input_all_cont['combined_string'][row]
    pdf_filename = input_all_cont['Site Number'][row]+'_Continuation_Sheet.pdf'
    generate_pdf(long_string, pdf_filename)

In [ ]:
# End of code